In [1]:
import os
import sys
import datetime
import math
import pandas as pd
import numpy as np
import quant_utils.data_moudle as dm
from quant_utils.constant import DB_CONN_JJTG_DATA, BARRA_SW21_FACTOR_NAME_DICT
from quant_utils.performance import Performance
import scipy.stats as stats

In [2]:
query_sql = """
WITH a AS (
	SELECT
		DATE_FORMAT( REPORT_DATE, "%Y%m%d" ) AS REPORT_DATE,
		TICKER_SYMBOL,
		NET_ASSET,
		NET_ASSET - lag( NET_ASSET, 1 ) over ( PARTITION BY TICKER_SYMBOL ORDER BY REPORT_DATE ) AS '环比',
		NET_ASSET - lag( NET_ASSET, 4 ) over ( PARTITION BY TICKER_SYMBOL ORDER BY REPORT_DATE ) AS "YOY" 
	FROM
		fund_asset_own 
	WHERE
		1 = 1 
		AND report_date >= '20100101' 
	) SELECT
	* 
FROM
	a 
WHERE
	REPORT_DATE REGEXP '0331|0630|0930|1231'
"""
df_asset = DB_CONN_JJTG_DATA.exec_query(query_sql)
fund_type_df = DB_CONN_JJTG_DATA.exec_query(
    """
    select DATE_FORMAT( REPORT_DATE, "%Y%m%d" ) AS REPORT_DATE, TICKER_SYMBOL, LEVEL_1
    from fund_type_own
    """
)

df_asset = df_asset.merge(fund_type_df)

map_dict = {
    i: dm.offset_trade_dt(i,0) for i in df_asset["REPORT_DATE"].unique()
}
df_asset["REPORT_DATE"] = df_asset["REPORT_DATE"].map(map_dict)

In [4]:
df_return_perf = DB_CONN_JJTG_DATA.exec_query(
    """
    SELECT
        DATE_FORMAT( a.TRADE_DT, "%Y%m%d" ) AS REPORT_DATE,
        TICKER_SYMBOL,
        F_AVGRETURN_QUARTER,
        F_AVGRETURN_HALFYEAR,
        F_AVGRETURN_YEAR,
        F_AVGRETURN_TWOYEAR,
        F_AVGRETURN_THREEYEAR,
        F_SHARPRATIO_HALFYEAR,
        F_SHARPRATIO_YEAR,
        F_SHARPRATIO_TWOYEAR,
        F_SHARPRATIO_THREEYEAR 
    FROM
        md_tradingdaynew b
        JOIN fund_performance a ON a.TRADE_DT = b.TRADE_DT 
    WHERE
        1 = 1 
        AND b.TRADE_DT >= '20100101' 
        AND b.SECU_MARKET = 83 
        AND b.IF_QUARTER_END = 1;
    """
)

In [5]:
data_set = df_asset.merge(df_return_perf)

In [7]:
result_list =[]
n_group = 5
for _, k in map_dict.items():
    for level in ["主动权益", "固收+", "固收", "权益指数", "债券指数", "FOF"]:
        temp_df = data_set.query(f"LEVEL_1 == '{level}' and REPORT_DATE=='{k}'")
        if temp_df.empty:
            continue 
        for x_name in ['环比', "YOY"]:
            for y_name in [
                "F_AVGRETURN_QUARTER", "F_AVGRETURN_HALFYEAR", "F_AVGRETURN_YEAR",
                "F_AVGRETURN_TWOYEAR", "F_AVGRETURN_THREEYEAR", "F_SHARPRATIO_HALFYEAR",
                "F_SHARPRATIO_YEAR", "F_SHARPRATIO_TWOYEAR", "F_SHARPRATIO_THREEYEAR"
            ]:
                temp = temp_df[[x_name, y_name]].dropna()
                if temp.empty:
                    continue
                temp["分组"] = pd.qcut(
                    temp[y_name], 
                    n_group, 
                    labels=range(1, n_group+1)
                )
                for i in range(1, n_group+1):
                    temp_1 = temp.query(f"分组=={i}")
                    stats_result = stats.spearmanr((temp_1[x_name]).rank(), temp_1[y_name].rank())
                    temp_result_list = [
                        k, level, x_name, y_name, i, 
                        stats_result.correlation, stats_result.pvalue
                    ]
                    temp_result_df = pd.DataFrame(temp_result_list).T
                    result_list.append(temp_result_df)
                stats_result = stats.spearmanr((temp[x_name]).rank(), temp[y_name].rank())
                temp_result_list = [
                        k, level, x_name, y_name, 'ALL', 
                        stats_result.correlation, stats_result.pvalue
                    ]
                temp_result_df = pd.DataFrame(temp_result_list).T
                result_list.append(temp_result_df)

C:\Users\Wilcoxon\AppData\Local\Temp\ipykernel_23404\4239264053.py:23: RuntimeWarning: Engine has switched to 'python' because numexpr does not support extension array dtypes. Please set your engine to python manually.
  temp_1 = temp.query(f"分组=={i}")
C:\Users\Wilcoxon\AppData\Local\Temp\ipykernel_23404\4239264053.py:23: RuntimeWarning: Engine has switched to 'python' because numexpr does not support extension array dtypes. Please set your engine to python manually.
  temp_1 = temp.query(f"分组=={i}")
C:\Users\Wilcoxon\AppData\Local\Temp\ipykernel_23404\4239264053.py:23: RuntimeWarning: Engine has switched to 'python' because numexpr does not support extension array dtypes. Please set your engine to python manually.
  temp_1 = temp.query(f"分组=={i}")
C:\Users\Wilcoxon\AppData\Local\Temp\ipykernel_23404\4239264053.py:23: RuntimeWarning: Engine has switched to 'python' because numexpr does not support extension array dtypes. Please set your engine to python manually.
  temp_1 = temp.query(

ValueError: Bin edges must be unique: array([0.249665, 0.249665, 0.249665, 0.249665, 0.249665, 0.249665]).
You can drop duplicate edges by setting the 'duplicates' kwarg

In [ ]:
result = pd.concat(result_list)
result.columns = ["日期", "分类", "规模分类", "指标", "分组", "相关性", "P_值"]
indicator_map = {
    "F_AVGRETURN_HALFYEAR":	'收益_6M',
"F_AVGRETURN_QUARTER":	'收益_3M',
'F_AVGRETURN_THREEYEAR':	'收益_3Y',
'F_AVGRETURN_TWOYEA':	'收益_2Y',
'F_AVGRETURN_YEAR':	'收益_1Y',
'F_SHARPRATIO_HALFYEAR':	'夏普比率_6M',
'F_SHARPRATIO_THREEYEAR':	'夏普比率_3Y',
'F_SHARPRATIO_TWOYEAR':	'夏普比率_2Y',
'F_SHARPRATIO_YEAR':	'夏普比率_1Y',
}
result["指标"] = result["指标"].map(indicator_map)
result.groupby(by=["分类", "规模分类", "指标", "分组"]).mean().to_excel("d:/规模变化_all.xlsx")

In [ ]:
result.to_excel("d:/规模变化.xlsx")